# 국장 & 미장 100위 크롤링

In [5]:
#필요한 라이브러리
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
# 크롤링할 대상 URL (한국 시가총액 상위 기업 목록)
url = "https://msg.soledot.com/finance/fo/sisemarketsumlist.sd#google_vignette"
# 사이트에 요청 보내고 HTML 받기
response = requests.get(url)
response

<Response [200]>

In [7]:
# 받은 HTML을 BeautifulSoup 객체로 파싱
soup = BeautifulSoup(response.text, "html.parser")
soup


<!--* TagLib -->
<!-- TagLib -->
<!--* Var  -->
<!-- Var -->
<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="soledot" name="author"/>
<meta content="실시간 시가총액 100위 종목정보를 제공하는 페이지입니다." name="description"/>
<meta content="시가총액,주식,상위거래,상위종목,실시간주식,실시간주가,코스피,코스닥,국내증시,상한가,하한가,종목순위,매도호가,매수호가,시가총액,현재가,전일비,등락률,종목명" name="keywords">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="#0c2770" name="theme-color"/>
<meta content="실시간 시가총액 100위 종목정보 - 무색계 블로그" property="og:title"/>
<meta content="실시간 시가총액 100위 종목정보를 제공하는 페이지입니다." property="og:description"/>
<meta content="https://msg.soledot.com/data/img/etc/msg.png" property="og:image"/>
<meta content="https://msg.soledot.com/finance/fo/sisemarketsumlist.sd" property="og:url"/>
<meta content="website" property="og:type"/>
<link href="https://msg.soledot.com/finance/fo/sisemarketsumlist.sd" rel="canonical"/>
<title>실시간 시가총액 100위 종

In [8]:
# 테이블에서 데이터 추출
rows = soup.select("table tbody tr")
rows



[<tr>
 <td class="text-center">1</td>
 <td class="text-left">삼성전자</td>
 <td class="text-right">
 																	63,300
 																</td>
 <td class="text-right text-primary">
 																	500
 																</td>
 <td class="text-right text-primary">-0.78%</td>
 <td class="text-right">
 																	100
 																</td>
 <td class="text-right">
 																	3,747,131
 																</td>
 <td class="text-right">
 																	5,919,638
 																</td>
 <td class="text-right">
 																	49.80
 																</td>
 <td class="text-right">
 																	23,689,699
 																</td>
 <td class="text-right">12.26</td>
 <td class="text-right">9.03</td>
 </tr>,
 <tr>
 <td class="text-center">2</td>
 <td class="text-left">SK하이닉스</td>
 <td class="text-right">
 																	270,500
 																</td>
 <td class="text-right text-primary">
 																	8,000
 																</td>
 <td class="text-right text-primary">-2.87%<

In [9]:
url = "https://companiesmarketcap.com/south-korea/largest-companies-in-south-korea-by-market-cap/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블의 tbody 안 tr 선택
rows = soup.select("table tbody tr")

data = []

for row in rows:
    rank_tag = row.select_one("td.rank-td")
    name_tag = row.select_one("div.company-name")
    ticker_tag = row.select_one("div.company-code")
    tds = row.select("td")
    
    if rank_tag and name_tag and ticker_tag and len(tds) >= 6:
        rank = rank_tag.text.strip()
        name = name_tag.text.strip()
        ticker = ticker_tag.text.strip()
        market_cap = tds[3].text.strip()
        price = tds[4].text.strip()
        today = tds[5].text.strip()
        
        data.append({
            "Rank": rank,
            "Ticker": ticker,
            "Name": name,
            "Market Cap": market_cap,
            "Price": price,
            "Today": today
        })

df_kr = pd.DataFrame(data)
print(df_kr)

   Rank     Ticker                Name Market Cap    Price  Today
0     1  005930.KS             Samsung  $306.36 B   $46.45  0.78%
1     2  000660.KS            SK Hynix  $137.01 B  $198.48  2.87%
2     3  207940.KS   Samsung Biologics   $54.20 B  $761.62  0.57%
3     4  373220.KS  LG Energy Solution   $53.31 B  $227.83  2.51%
4     5      HYMTF             Hyundai   $38.88 B   $53.00  0.00%
..  ...        ...                 ...        ...      ...    ...
95   96  000080.KS           HiteJinro    $1.05 B   $15.30  1.18%
96   97  073240.KS          Kumho Tire    $1.03 B    $3.61  0.51%
97   98  007310.KS              Ottogi    $1.01 B  $293.86  1.72%
98   99  137310.KS        SD BioSensor    $0.89 B    $7.45  3.05%
99  100  192080.KS        DoubleUGames    $0.82 B   $41.60  1.90%

[100 rows x 6 columns]


In [10]:
df_kr['Rank'].value_counts()

Rank
1      1
2      1
3      1
4      1
5      1
      ..
96     1
97     1
98     1
99     1
100    1
Name: count, Length: 100, dtype: int64

In [11]:
df_kr[df_kr['Rank'] == 1]

,Rank,Ticker,Name,Market Cap,Price,Today


- 코스피와 코스닥 TOP100이 전부 크롤링된것을 확인 -> 코스피만 필요하기 때문에 코스닥 데이터는 전처리 진행

In [12]:
df_kr = df_kr[:100]
df_kr

,Rank,Ticker,Name,Market Cap,Price,Today
0,1,005930.KS,Samsung,$306.36 B,$46.45,0.78%
1,2,000660.KS,SK Hynix,$137.01 B,$198.48,2.87%
2,3,207940.KS,Samsung Biologics,$54.20 B,$761.62,0.57%
3,4,373220.KS,LG Energy Solution,$53.31 B,$227.83,2.51%
4,5,HYMTF,Hyundai,$38.88 B,$53.00,0.00%
...,...,...,...,...,...,...
95,96,000080.KS,HiteJinro,$1.05 B,$15.30,1.18%
96,97,073240.KS,Kumho Tire,$1.03 B,$3.61,0.51%
97,98,007310.KS,Ottogi,$1.01 B,$293.86,1.72%
98,99,137310.KS,SD BioSensor,$0.89 B,$7.45,3.05%


# 미국 주식장

In [13]:
url1 = 'https://companiesmarketcap.com/'

response = requests.get(url1)
soup = BeautifulSoup(response.text, "html.parser")

rows = soup.select("table tbody tr")

data = []

for row in rows:
    rank_tag = row.select_one("td.rank-td")
    name_tag = row.select_one("div.company-name")
    ticker_tag = row.select_one("div.company-code")
    tds = row.select("td")
    
    if rank_tag and name_tag and len(tds) >= 6:
        rank = rank_tag.text.strip()
        name = name_tag.text.strip()
        Ticker = ticker_tag.text.strip()
        market_cap = tds[3].text.strip()
        price = tds[4].text.strip()
        today = tds[5].text.strip()
        
        data.append({
            "Rank": rank,
            "Ticker" : Ticker,
            "Name": name,
            "Market Cap": market_cap,
            "Price": price,
            "Today": today

        })

df_gb = pd.DataFrame(data)

print(df_gb)  # 상위 10개 출력


   Rank   Ticker               Name Market Cap    Price  Today
0     1     NVDA             NVIDIA   $3.885 T  $159.34  1.33%
1     2     MSFT          Microsoft   $3.707 T  $498.84  1.58%
2     3     AAPL              Apple   $3.189 T  $213.55  0.52%
3     4     AMZN             Amazon   $2.371 T  $223.41  1.59%
4     5     GOOG  Alphabet (Google)   $2.184 T  $180.55  0.44%
..  ...      ...                ...        ...      ...    ...
95   96      ARM       Arm Holdings  $164.28 B  $155.09  0.30%
96   97  3968.HK            CM Bank  $163.00 B    $6.61  1.14%
97   98       BA             Boeing  $162.80 B  $215.92  1.83%
98   99   AIR.PA             Airbus  $162.45 B  $206.34  0.92%
99  100     SPGI         S&P Global  $162.42 B  $529.61  0.76%

[100 rows x 6 columns]


- 미장이 미국주시장만 뜻하는건지 아니면 전세계 주식장을 뜻하는지 애매해서 url만 바꾸면 되게 설정
- 미국 주식 url : 'https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/' 


### 각 주식의 주요테마 추가

In [14]:
import yfinance as yf

In [20]:
from tqdm import tqdm
tqdm.pandas()

# 세 개 열 추가용 함수
def get_company_info(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        sector = info.get("sector", "N/A")
        industry = info.get("industry", "N/A")
        summary = info.get("longBusinessSummary", "N/A")

        return pd.Series([sector, industry, summary])
    
    except Exception as e:
        return pd.Series(["N/A", "N/A", "N/A"])

# 실제 사용 (예: korea_df)
df_kr[["Sector", "Industry", "Summary"]] = df_kr["Ticker"].progress_apply(get_company_info)

#global_df에도 동일하게 적용 가능
df_gb[["Sector", "Industry", "Summary"]] = df_gb["Ticker"].progress_apply(get_company_info)

100%|██████████| 100/100 [00:37<00:00,  2.66it/s]


In [21]:
df_kr

,Rank,Ticker,Name,Market Cap,Price,Today,Sector,Industry,Summary
0,1,005930.KS,Samsung,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
1,2,000660.KS,SK Hynix,$137.01 B,$198.48,2.87%,Technology,Semiconductors,"SK hynix Inc., together with its subsidiaries,..."
2,3,207940.KS,Samsung Biologics,$54.20 B,$761.62,0.57%,Healthcare,Biotechnology,"Samsung Biologics Co.,Ltd., together with its ..."
3,4,373220.KS,LG Energy Solution,$53.31 B,$227.83,2.51%,Industrials,Electrical Equipment & Parts,"LG Energy Solution, Ltd. provides energy solut..."
4,5,HYMTF,Hyundai,$38.88 B,$53.00,0.00%,Consumer Cyclical,Auto Manufacturers,"Hyundai Motor Company, together with its subsi..."
...,...,...,...,...,...,...,...,...,...
95,96,000080.KS,HiteJinro,$1.05 B,$15.30,1.18%,Consumer Defensive,Beverages - Wineries & Distilleries,"HITEJINRO Co., Ltd. manufactures and sells alc..."
96,97,073240.KS,Kumho Tire,$1.03 B,$3.61,0.51%,Consumer Cyclical,Auto Parts,"Kumho Tire Co., Inc., engages in the manufactu..."
97,98,007310.KS,Ottogi,$1.01 B,$293.86,1.72%,Consumer Defensive,Packaged Foods,Otoki Corporation produces and sells various f...
98,99,137310.KS,SD BioSensor,$0.89 B,$7.45,3.05%,Healthcare,Medical Instruments & Supplies,"SD Biosensor, Inc, an in-vitro diagnostic comp..."


In [22]:
df_gb

,Rank,Ticker,Name,Market Cap,Price,Today,Sector,Industry,Summary
0,1,NVDA,NVIDIA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
1,2,MSFT,Microsoft,$3.707 T,$498.84,1.58%,Technology,Software - Infrastructure,Microsoft Corporation develops and supports so...
2,3,AAPL,Apple,$3.189 T,$213.55,0.52%,Technology,Consumer Electronics,"Apple Inc. designs, manufactures, and markets ..."
3,4,AMZN,Amazon,$2.371 T,$223.41,1.59%,Consumer Cyclical,Internet Retail,"Amazon.com, Inc. engages in the retail sale of..."
4,5,GOOG,Alphabet (Google),$2.184 T,$180.55,0.44%,Communication Services,Internet Content & Information,Alphabet Inc. offers various products and plat...
...,...,...,...,...,...,...,...,...,...
95,96,ARM,Arm Holdings,$164.28 B,$155.09,0.30%,Technology,Semiconductors,"Arm Holdings plc architects, develops, and lic..."
96,97,3968.HK,CM Bank,$163.00 B,$6.61,1.14%,Financial Services,Banks - Regional,"China Merchants Bank Co., Ltd., together with ..."
97,98,BA,Boeing,$162.80 B,$215.92,1.83%,Industrials,Aerospace & Defense,"The Boeing Company, together with its subsidia..."
98,99,AIR.PA,Airbus,$162.45 B,$206.34,0.92%,Industrials,Aerospace & Defense,"Airbus SE, together with its subsidiaries, eng..."


### 야후 API를 이용하여 두달간의 주가 저장 - 국내

In [23]:
#!pip install yfinance

In [24]:
#어떤식으로 나오나 확인하는 코드 
#ticker = yf.Ticker("219480.KS")  # 삼성전자
#hist = ticker.history(period="2mo", interval="4h")  # 최근 1개월, 4시간 간격
#hist


In [25]:
#리스트를 만들어 데이터프레임을 추가하는 코드
korea_history = []
for i in range(len(df_kr)):
    name = df_kr['Name'][i]
    Ticker = df_kr['Ticker'][i]
    ticker = yf.Ticker(Ticker)
    hist = ticker.history(period="2mo", interval="4h")
    hist['name'] = name
    hist['Ticker'] = Ticker
    korea_history.append(hist)
korea_history

[                              Open     High      Low    Close    Volume  \
 Datetime                                                                  
 2025-05-07 09:00:00+09:00  54500.0  55000.0  54300.0  54300.0   4209646   
 2025-05-07 13:00:00+09:00  54350.0  54500.0  54300.0  54450.0   2880602   
 2025-05-08 09:00:00+09:00  55100.0  55500.0  54700.0  54900.0   7738123   
 2025-05-08 13:00:00+09:00  54900.0  55000.0  54600.0  54650.0   2831741   
 2025-05-09 09:00:00+09:00  54700.0  54900.0  54400.0  54650.0   4073238   
 ...                            ...      ...      ...      ...       ...   
 2025-07-02 13:00:00+09:00  60600.0  60800.0  60500.0  60550.0   2536969   
 2025-07-03 09:00:00+09:00  61300.0  62700.0  61100.0  62550.0  18238557   
 2025-07-03 13:00:00+09:00  62550.0  63500.0  62550.0  63500.0   8193679   
 2025-07-04 09:00:00+09:00  64700.0  64700.0  63000.0  63250.0  16355287   
 2025-07-04 13:00:00+09:00  63300.0  63700.0  63000.0  63350.0   4462836   
 
          

In [26]:
korea_df = pd.DataFrame()
korea_df = pd.concat(korea_history, ignore_index=False)
korea_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker
Datetime,,,,,,,,,
2025-05-07 09:00:00+09:00,54500.0,55000.0,54300.0,54300.0,4209646,0.0,0.0,Samsung,005930.KS
2025-05-07 13:00:00+09:00,54350.0,54500.0,54300.0,54450.0,2880602,0.0,0.0,Samsung,005930.KS
2025-05-08 09:00:00+09:00,55100.0,55500.0,54700.0,54900.0,7738123,0.0,0.0,Samsung,005930.KS
2025-05-08 13:00:00+09:00,54900.0,55000.0,54600.0,54650.0,2831741,0.0,0.0,Samsung,005930.KS
2025-05-09 09:00:00+09:00,54700.0,54900.0,54400.0,54650.0,4073238,0.0,0.0,Samsung,005930.KS
...,...,...,...,...,...,...,...,...,...
2025-07-02 13:00:00+09:00,56900.0,58100.0,56800.0,58000.0,23933,0.0,0.0,DoubleUGames,192080.KS
2025-07-03 09:00:00+09:00,58300.0,59000.0,57500.0,58400.0,45858,0.0,0.0,DoubleUGames,192080.KS
2025-07-03 13:00:00+09:00,58400.0,58400.0,57300.0,58000.0,42751,0.0,0.0,DoubleUGames,192080.KS


In [27]:
# 인덱스 초기화 (Datetime을 일반 열로)
korea_df = korea_df.reset_index()

# 'name'과 '회사명' 기준으로 병합
korea_df = pd.merge(korea_df, df_kr, left_on='name', right_on='Name', how='left')

# 병합 후 필요 없는 중복 열 제거 (예: 회사명)
korea_df = korea_df.drop(columns=['Name'])

# 다시 Datetime을 인덱스로 설정하려면:
korea_df = korea_df.set_index('Datetime')


In [28]:
korea_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker_x,Rank,Ticker_y,Market Cap,Price,Today,Sector,Industry,Summary
Datetime,,,,,,,,,,,,,,,,,
2025-05-07 09:00:00+09:00,54500.0,55000.0,54300.0,54300.0,4209646,0.0,0.0,Samsung,005930.KS,1,005930.KS,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-07 13:00:00+09:00,54350.0,54500.0,54300.0,54450.0,2880602,0.0,0.0,Samsung,005930.KS,1,005930.KS,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-08 09:00:00+09:00,55100.0,55500.0,54700.0,54900.0,7738123,0.0,0.0,Samsung,005930.KS,1,005930.KS,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-08 13:00:00+09:00,54900.0,55000.0,54600.0,54650.0,2831741,0.0,0.0,Samsung,005930.KS,1,005930.KS,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-09 09:00:00+09:00,54700.0,54900.0,54400.0,54650.0,4073238,0.0,0.0,Samsung,005930.KS,1,005930.KS,$306.36 B,$46.45,0.78%,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 13:00:00+09:00,56900.0,58100.0,56800.0,58000.0,23933,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.82 B,$41.60,1.90%,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
2025-07-03 09:00:00+09:00,58300.0,59000.0,57500.0,58400.0,45858,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.82 B,$41.60,1.90%,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
2025-07-03 13:00:00+09:00,58400.0,58400.0,57300.0,58000.0,42751,0.0,0.0,DoubleUGames,192080.KS,100,192080.KS,$0.82 B,$41.60,1.90%,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."


In [29]:
#필요한 열 선택, 보기쉽게 재배치
korea_df = korea_df[['Rank','Ticker_x', 'name', 'Open', 'High', 'Low', 'Close', 'Volume',"Sector", "Industry", "Summary"]]
korea_df.columns = ['현재 시총 순위', '종목 코드', '기업명', '시가', '고가', '저가', '종가', '거래량',"산업군", "업종", "기업 요약"]
korea_df

,현재 시총 순위,종목 코드,기업명,시가,고가,저가,종가,거래량,산업군,업종,기업 요약
Datetime,,,,,,,,,,,
2025-05-07 09:00:00+09:00,1,005930.KS,Samsung,54500.0,55000.0,54300.0,54300.0,4209646,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-07 13:00:00+09:00,1,005930.KS,Samsung,54350.0,54500.0,54300.0,54450.0,2880602,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-08 09:00:00+09:00,1,005930.KS,Samsung,55100.0,55500.0,54700.0,54900.0,7738123,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-08 13:00:00+09:00,1,005930.KS,Samsung,54900.0,55000.0,54600.0,54650.0,2831741,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2025-05-09 09:00:00+09:00,1,005930.KS,Samsung,54700.0,54900.0,54400.0,54650.0,4073238,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 13:00:00+09:00,100,192080.KS,DoubleUGames,56900.0,58100.0,56800.0,58000.0,23933,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
2025-07-03 09:00:00+09:00,100,192080.KS,DoubleUGames,58300.0,59000.0,57500.0,58400.0,45858,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
2025-07-03 13:00:00+09:00,100,192080.KS,DoubleUGames,58400.0,58400.0,57300.0,58000.0,42751,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."


### 야후 API를 이용하여 두달간의 주가 저장 - 전세계

In [30]:
global_history = []

for i in range(len(df_gb)):
    name = df_gb['Name'][i]
    Ticker = df_gb['Ticker'][i]
    ticker = yf.Ticker(Ticker)
    hist = ticker.history(period="2mo", interval="4h")
    hist['name'] = name
    hist['Ticker'] = Ticker
    global_history.append(hist)

global_history


HTTP Error 404: 
$IHC.AE: possibly delisted; no price data found  (period=2mo) (Yahoo error = "No data found, symbol may be delisted")


[                                 Open        High         Low       Close  \
 Datetime                                                                    
 2025-05-05 09:30:00-04:00  112.904999  114.379997  112.660004  114.129997   
 2025-05-05 13:30:00-04:00  114.135002  114.660004  113.669998  113.809998   
 2025-05-06 09:30:00-04:00  111.639999  113.790001  110.821999  112.919998   
 2025-05-06 13:30:00-04:00  112.930000  114.739998  112.720001  113.445000   
 2025-05-07 09:30:00-04:00  113.044998  114.220001  112.614998  113.945000   
 ...                               ...         ...         ...         ...   
 2025-07-01 09:30:00-04:00  156.259995  157.199997  151.490005  154.274994   
 2025-07-01 13:30:00-04:00  154.330002  154.880005  152.910095  153.279999   
 2025-07-02 09:30:00-04:00  153.029999  157.600006  152.979996  157.205399   
 2025-07-02 13:30:00-04:00  157.205002  157.429993  156.509995  157.240005   
 2025-07-03 09:30:00-04:00  158.350006  160.979996  157.770004  

In [31]:
len(global_history)

100

In [32]:
global_df = pd.DataFrame()
global_df = pd.concat(global_history)
global_df


,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker,Adj Close
2025-05-05 09:30:00-04:00,112.904999,114.379997,112.660004,114.129997,79898207.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-05 13:30:00-04:00,114.135002,114.660004,113.669998,113.809998,32525343.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-06 09:30:00-04:00,111.639999,113.790001,110.821999,112.919998,91150436.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-06 13:30:00-04:00,112.930000,114.739998,112.720001,113.445000,43956454.0,0.0,0.0,NVIDIA,NVDA,NaN
2025-05-07 09:30:00-04:00,113.044998,114.220001,112.614998,113.945000,77486400.0,0.0,0.0,NVIDIA,NVDA,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-07-01 09:30:00-04:00,525.169983,529.070007,522.990112,529.070007,426620.0,0.0,0.0,S&P Global,SPGI,NaN
2025-07-01 13:30:00-04:00,529.200012,530.080017,527.354980,529.250000,460653.0,0.0,0.0,S&P Global,SPGI,NaN
2025-07-02 09:30:00-04:00,527.520020,528.380005,524.030029,526.318970,295609.0,0.0,0.0,S&P Global,SPGI,NaN
2025-07-02 13:30:00-04:00,526.210022,526.330017,522.726013,525.645020,409095.0,0.0,0.0,S&P Global,SPGI,NaN


In [33]:
# 인덱스 초기화 (Datetime을 일반 열로)
global_df = global_df.reset_index()

# 'name'과 '회사명' 기준으로 병합
global_df = pd.merge(global_df, df_gb, left_on='name', right_on='Name', how='left')

# 병합 후 필요 없는 중복 열 제거 (예: 회사명)
global_df = global_df.drop(columns=['Name'])

global_df

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,name,Ticker_x,Adj Close,Rank,Ticker_y,Market Cap,Price,Today,Sector,Industry,Summary
0,2025-05-05 09:30:00-04:00,112.904999,114.379997,112.660004,114.129997,79898207.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
1,2025-05-05 13:30:00-04:00,114.135002,114.660004,113.669998,113.809998,32525343.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
2,2025-05-06 09:30:00-04:00,111.639999,113.790001,110.821999,112.919998,91150436.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
3,2025-05-06 13:30:00-04:00,112.930000,114.739998,112.720001,113.445000,43956454.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
4,2025-05-07 09:30:00-04:00,113.044998,114.220001,112.614998,113.945000,77486400.0,0.0,0.0,NVIDIA,NVDA,NaN,1,NVDA,$3.885 T,$159.34,1.33%,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,2025-07-01 09:30:00-04:00,525.169983,529.070007,522.990112,529.070007,426620.0,0.0,0.0,S&P Global,SPGI,NaN,100,SPGI,$162.42 B,$529.61,0.76%,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8695,2025-07-01 13:30:00-04:00,529.200012,530.080017,527.354980,529.250000,460653.0,0.0,0.0,S&P Global,SPGI,NaN,100,SPGI,$162.42 B,$529.61,0.76%,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8696,2025-07-02 09:30:00-04:00,527.520020,528.380005,524.030029,526.318970,295609.0,0.0,0.0,S&P Global,SPGI,NaN,100,SPGI,$162.42 B,$529.61,0.76%,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8697,2025-07-02 13:30:00-04:00,526.210022,526.330017,522.726013,525.645020,409095.0,0.0,0.0,S&P Global,SPGI,NaN,100,SPGI,$162.42 B,$529.61,0.76%,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."


In [34]:
#필요한 열 선택, 보기쉽게 재배치
global_df = global_df[['index','Rank','Ticker_x', 'name', 'Open', 'High', 'Low', 'Close', 'Volume',"Sector", "Industry", "Summary"]]
global_df.columns = ['Datetime','현재 시총 순위', '종목 코드', '기업명', '시가', '고가', '저가', '종가', '거래량',"산업군", "업종", "기업 요약"]
global_df

,Datetime,현재 시총 순위,종목 코드,기업명,시가,고가,저가,종가,거래량,산업군,업종,기업 요약
0,2025-05-05 09:30:00-04:00,1,NVDA,NVIDIA,112.904999,114.379997,112.660004,114.129997,79898207.0,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
1,2025-05-05 13:30:00-04:00,1,NVDA,NVIDIA,114.135002,114.660004,113.669998,113.809998,32525343.0,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
2,2025-05-06 09:30:00-04:00,1,NVDA,NVIDIA,111.639999,113.790001,110.821999,112.919998,91150436.0,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
3,2025-05-06 13:30:00-04:00,1,NVDA,NVIDIA,112.930000,114.739998,112.720001,113.445000,43956454.0,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
4,2025-05-07 09:30:00-04:00,1,NVDA,NVIDIA,113.044998,114.220001,112.614998,113.945000,77486400.0,Technology,Semiconductors,"NVIDIA Corporation, a computing infrastructure..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8694,2025-07-01 09:30:00-04:00,100,SPGI,S&P Global,525.169983,529.070007,522.990112,529.070007,426620.0,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8695,2025-07-01 13:30:00-04:00,100,SPGI,S&P Global,529.200012,530.080017,527.354980,529.250000,460653.0,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8696,2025-07-02 09:30:00-04:00,100,SPGI,S&P Global,527.520020,528.380005,524.030029,526.318970,295609.0,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."
8697,2025-07-02 13:30:00-04:00,100,SPGI,S&P Global,526.210022,526.330017,522.726013,525.645020,409095.0,Financial Services,Financial Data & Stock Exchanges,"S&P Global Inc., together with its subsidiarie..."


In [35]:
korea_df=korea_df.reset_index()
korea_df

,Datetime,현재 시총 순위,종목 코드,기업명,시가,고가,저가,종가,거래량,산업군,업종,기업 요약
0,2025-05-07 09:00:00+09:00,1,005930.KS,Samsung,54500.0,55000.0,54300.0,54300.0,4209646,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
1,2025-05-07 13:00:00+09:00,1,005930.KS,Samsung,54350.0,54500.0,54300.0,54450.0,2880602,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
2,2025-05-08 09:00:00+09:00,1,005930.KS,Samsung,55100.0,55500.0,54700.0,54900.0,7738123,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
3,2025-05-08 13:00:00+09:00,1,005930.KS,Samsung,54900.0,55000.0,54600.0,54650.0,2831741,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
4,2025-05-09 09:00:00+09:00,1,005930.KS,Samsung,54700.0,54900.0,54400.0,54650.0,4073238,Technology,Consumer Electronics,"Samsung Electronics Co., Ltd. engages in the c..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8184,2025-07-02 13:00:00+09:00,100,192080.KS,DoubleUGames,56900.0,58100.0,56800.0,58000.0,23933,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
8185,2025-07-03 09:00:00+09:00,100,192080.KS,DoubleUGames,58300.0,59000.0,57500.0,58400.0,45858,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
8186,2025-07-03 13:00:00+09:00,100,192080.KS,DoubleUGames,58400.0,58400.0,57300.0,58000.0,42751,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
8187,2025-07-04 09:00:00+09:00,100,192080.KS,DoubleUGames,57900.0,58100.0,56300.0,57300.0,50022,Communication Services,Electronic Gaming & Multimedia,"DoubleUGames Co., Ltd. develops and services m..."
